In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
(img_train,label_train),(img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [3]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [4]:
#definicja modelu
#model typu hipermodel(model stosowany do hipertuningu)
#hipermodel -> 1 korzystanie z funkcju konstruktora modeli
#-> poprzez tworzenie pochodnych klasy HyperModel

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    #dostrojenie gęstości warstwy -> liczby neuronów
    hp_units = hp.Int('units',min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    #dostrojenie algorytmu optymalizującego (learning_rate)
    hp_learning_rate = hp.Choice('learnig_rate',values=[1e-2,1e-3,1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [5]:
#Utworzenie wystąpienie Tunera i HyperTuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='into_to_kt')

In [8]:
#wywołanie zwrotne, wstrzymanie treningu w przypadku istotnej utraty wartości walidacji
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 5)

In [9]:
#wyszukiwanie hiperparametrów
tuner.search(img_train, label_train, epochs=50, validation_split = 0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Wyszukiwanie hiperparametrów zakończone. 
Optymalna wartość unitów dla pierwszej epoki: {best_hps.get('units')},
optymalna wartość learning_rate dla optymalizatora wynosi: {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 03m 23s]
val_accuracy: 0.8630833625793457

Best val_accuracy So Far: 0.893583357334137
Total elapsed time: 00h 28m 07s
INFO:tensorflow:Oracle triggered exit


KeyError: KeyError: 'learning_rate does not exist.'

In [ ]:
#Trenowanie modelu
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print('Najlepsza epoka: %d' %(best_epoch,))

Epoch 17/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1724 - accuracy: 0.9339 - val_loss: 0.3203 - val_accuracy: 0.8958
Epoch 18/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1675 - accuracy: 0.9372 - val_loss: 0.3330 - val_accuracy: 0.8958
Epoch 19/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.1592 - accuracy: 0.9416 - val_loss: 0.3385 - val_accuracy: 0.8946
Epoch 20/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1561 - accuracy: 0.9415 - val_loss: 0.3541 - val_accuracy: 0.8925
Epoch 21/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.1554 - accuracy: 0.9419 - val_loss: 0.3284 - val_accuracy: 0.8988
Epoch 22/50
1500/1500 [==============================] - 33s 22ms/step - loss: 0.1474 - accuracy: 0.9445 - val_loss: 0.3364 - val_accuracy: 0.8982
Epoch 23/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.1412 - accuracy: 0.9473 - val_loss: 0.3747 - 

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train,label_train,epochs=best_epoch,validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(img_test,label_test)
print(f"[test loss, test accuracy]:{eval_result}")